In [ ]:
pip install markitdown neo4j-graphrag neo4j openai yfiles_jupyter_graphs_for_neo4j

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [ ]:
from markitdown import MarkItDown
from openai import OpenAI

client = OpenAI()
md = MarkItDown(llm_client=client, llm_model="gpt-4o")
result = md.convert("https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q4-2024-Update.pdf")

In [ ]:
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings as Embeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation.graphrag import GraphRAG
from neo4j_graphrag.indexes import create_vector_index
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver("neo4j+s://7ffc03b9.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

ex_llm=LLM(
   model_name="gpt-4o-mini",
   model_params={
       "response_format": {"type": "json_object"},
       "temperature": 0
   })

embedder = Embeddings()

# 1. Build KG and Store in Neo4j Database
kg_builder = SimpleKGPipeline(
   llm=ex_llm,
   driver=driver,
   embedder=embedder,
   from_pdf=False
)
await kg_builder.run_async(text=result.text_content)

create_vector_index(driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

# 2. KG Retriever
vector_retriever = VectorRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder
)

# 3. GraphRAG Class
llm = LLM(model_name="gpt-4o")
rag = GraphRAG(llm=llm, retriever=vector_retriever)

/usr/lib/python3.11/asyncio/events.py:84: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)
<frozen importlib._bootstrap>:673: ImportWarning: APICoreClientInfoImportHook.exec_module() not found; falling back to load_module()


In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

/usr/local/lib/python3.11/dist-packages/neo4j/_sync/work/workspace.py:82: ResourceWarning: unclosed  Session: <neo4j._sync.work.session.Session object at 0x7b44178a3010>.
  _unclosed_resource_warn(self)
/usr/local/lib/python3.11/dist-packages/neo4j/_sync/work/workspace.py:87: DeprecationWarning: Relying on Session's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close sessions automatically.
  _deprecation_warn(


GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
response = rag.search("Summarize the data")
print(response.answer)

/usr/local/lib/python3.11/dist-packages/neo4j_graphrag/generation/graphrag.py:116: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/neo4j_graphrag/retrievers/vector.py:196: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(


The provided data includes financial and operational highlights for Tesla as well as some details on assets, liabilities, and cash flow. Key points include:

- Financial Highlights: Tesla achieved $7.1 billion in GAAP operating income for 2024 and $1.6 billion in Q4. GAAP net income for 2024 was $7.1 billion, with $2.3 billion in Q4 (including a $0.6 billion gain on digital assets). Non-GAAP net income for 2024 was $8.4 billion, and $2.6 billion in Q4. 

- Cash Flow: Operating cash flow was $14.9 billion for 2024, with $4.8 billion in Q4. Free cash flow was $3.6 billion for 2024 and $2.0 billion in Q4. Cash and investments increased by $7.5 billion to $36.6 billion by the end of the year.

- Operations: AI training compute increased by over 400% in 2024. Model Y is expected to be the best-selling vehicle globally for 2024, with enhancements and launches across all markets. Tesla invested significantly in infrastructure for future growth, including new vehicle manufacturing capabilities

In [ ]:
response = rag.search("What are some of the non EV revenue streams for Tesla in 2024?")
print(response.answer)

In 2024, Tesla's non-EV revenue streams included its energy generation and storage business, which achieved record deployments and gross profit. The company also focused on expanding its capacity for Megapack and Powerwall products. Additionally, Tesla's "Services and Other" sector, which supports new vehicle sales, reported its third consecutive year of profitability. These business activities contributed significantly to Tesla's overall revenue beyond its electric vehicle sales.
